# Poisson Example 2D

Authors: Kidus Teshome, Cameron Seebeck, Cian Wilson

## Description

As a reminder, in this case we are seeking the approximate solution to
\begin{equation}
- \nabla^2 T = -\tfrac{5}{4} \exp \left( x+\tfrac{y}{2} \right)
\end{equation}
in a unit square, $\Omega=[0,1]\times[0,1]$, imposing the boundary conditions
\begin{align}
  T &= \exp\left(x+\tfrac{y}{2}\right) && \text{on } \partial\Omega \text{ where } x=0 \text{ or } y=0 \\
  \nabla T\cdot \hat{\vec{n}} &= \exp\left(x + \tfrac{y}{2}\right) && \text{on } \partial\Omega \text{ where } x=1  \\
  \nabla T\cdot \hat{\vec{n}} &= \tfrac{1}{2}\exp\left(x + \tfrac{y}{2}\right) && \text{on } \partial\Omega \text{ where } y=1
 \end{align}

The analytical solution to this problem is $T(x,y) = \exp\left(x+\tfrac{y}{2}\right)$.

## Themes and variations

* Given that we know the exact solution to this problem is $T(x,y)$=$\exp\left(x+\tfrac{y}{2}\right)$ write a python function to evaluate the error in our numerical solution.
* Loop over a variety of numbers of elements, `ne`, and polynomial degrees, `p`, and check that the numerical solution converges with an increasing number of degrees of freedom.
* Write an equation for the gradient of $\tilde{T}$, describe it using UFL, solve it, and plot the solution.

### Preamble

Start by loading `solve_poisson_2d` from `notebooks/poisson_2d.ipynb` and setting up some paths.

In [1]:
import ipyparallel as ipp
np = 4
rc = ipp.Cluster(engine_launcher_class="mpi", n=np).start_and_connect_sync()

Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/4 [00:00<?, ?engine/s]

mpiexec error output:
-------------------------------------------------------------------------------------------------------
Build dofmap data                                                           |     2  0.028485  0.056970
Build sparsity                                                              |     1  0.039056  0.039056
Compute connectivity 1-0                                                    |     1  0.002149  0.002149
Compute dof reordering map                                                  |     2  0.001772  0.003543
Compute entities of dim = 1                                                 |     1  0.043096  0.043096
Compute graph partition (SCOTCH)                                            |     1  0.172813  0.172813
Compute local part of mesh dual graph                                       |     2  0.106380  0.212761
Compute local-to-local map                                                  |     1  0.009454  0.009454
Compute non-local part of mesh dual graph 

In [2]:
%%px
from poisson_2d import solve_poisson_2d
from mpi4py import MPI
import numpy as np
import ufl
import sys, os
basedir = ''
if "__file__" in globals(): basedir = os.path.dirname(__file__)
sys.path.append(os.path.join(basedir, os.path.pardir, 'python'))
import utils
import matplotlib.pyplot as pl
import pyvista as pv
if __name__ == "__main__" and "__file__" in globals():
    pv.OFF_SCREEN = True
import pathlib
if __name__ == "__main__":
    output_folder = pathlib.Path(os.path.join(basedir, "output"))
    output_folder.mkdir(exist_ok=True, parents=True)
import time

%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

In [3]:
%%px
print(MPI.COMM_WORLD.rank, MPI.COMM_WORLD.size)

[stdout:1] 1 4


[stdout:0] 0 4


[stdout:3] 3 4


[stdout:2] 2 4


In [4]:
%%px
if __name__ == "__main__":
    ne = 640
    p = 2
    # Solve the 2D Poisson problem
    start_time = time.time()
    T_i = solve_poisson_2d(ne, p)
    end_time = time.time()
    comm = T_i.function_space.mesh.comm
    print(f"{comm.rank} ({T_i.function_space.dofmap.index_map.size_local}) - time taken: {end_time - start_time} seconds")

[stdout:1] Generate mesh (1) 0.7812075614929199
Generate functionspace (1) 0.09311032295227051
Generate functions (1) 5.412101745605469e-05
Generate Dirichlet bcs (1) 0.04608035087585449
Generate Neumann bcs (1) 0.0005326271057128906
Generate forms (1) 0.00896000862121582
Assemble problem (1) 0.3468780517578125
Solve problem (1) 10.798412084579468
1 (410114) - time taken: 12.07573413848877 seconds


[stdout:0] Generate mesh (0) 0.781559944152832
Generate functionspace (0) 0.09321904182434082
Generate functions (0) 0.0001461505889892578
Generate Dirichlet bcs (0) 0.036893367767333984
Generate Neumann bcs (0) 0.0004787445068359375
Generate forms (0) 0.017628908157348633
Assemble problem (0) 0.347092866897583
Solve problem (0) 10.800966739654541
0 (410577) - time taken: 12.078472375869751 seconds

[MPI_MAX] Summary of timings                                                |  reps  wall avg  wall tot
-------------------------------------------------------------------------------------------------------
Build dofmap data                                                           |     2  0.028485  0.056970
Build sparsity                                                              |     1  0.039056  0.039056
Compute connectivity 1-0                                                    |     1  0.002149  0.002149
Compute dof reordering map                                                  |

[stdout:3] Generate mesh (3) 0.7805020809173584
Generate functionspace (3) 0.09324145317077637
Generate functions (3) 5.6743621826171875e-05
Generate Dirichlet bcs (3) 0.04704904556274414
Generate Neumann bcs (3) 0.0004742145538330078
Generate forms (3) 0.01034998893737793
Assemble problem (3) 0.3444197177886963
Solve problem (3) 10.79893445968628
3 (409815) - time taken: 12.075514554977417 seconds


[stdout:2] Generate mesh (2) 0.7809104919433594
Generate functionspace (2) 0.09273457527160645
Generate functions (2) 6.389617919921875e-05
Generate Dirichlet bcs (2) 0.047963857650756836
Generate Neumann bcs (2) 0.0004975795745849609
Generate forms (2) 0.008292198181152344
Assemble problem (2) 0.34354662895202637
Solve problem (2) 10.801195859909058
2 (410455) - time taken: 12.075725078582764 seconds


%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

In [ ]:
%%px
if __name__ == "__main__":
    # plot the solution as a colormap
    plotter = utils.plot_scalar(T_i, gather=True)
    # save the plot
    utils.plot_save(plotter, output_folder / "2d_poisson_test_single_solution_ipyparallel_4_ne{:d}.png".format(ne,))
    comm = T_i.function_space.mesh.comm
    if comm.size > 1:
        # if we're running in parallel (e.g. from a script) then save an image per process as well
        plotter_p = utils.plot_scalar(T_i)
        utils.plot_save(plotter_p, output_folder / "2d_poisson_test_single_solution_ipyparallel_4_ne{:d}_p{:d}.png".format(ne,comm.rank,))

%px:   0%|          | 0/4 [00:23<?, ?tasks/s]

CompositeError: one or more exceptions from call to method: %px
[Engine Exception]EngineError: Engine 0 died while running task '23c0bf81-13be16aa1d2cc66632185709_70881_13'
[Engine Exception]EngineError: Engine 1 died while running task '23c0bf81-13be16aa1d2cc66632185709_70881_14'
[Engine Exception]EngineError: Engine 2 died while running task '23c0bf81-13be16aa1d2cc66632185709_70881_15'
[Engine Exception]EngineError: Engine 3 died while running task '23c0bf81-13be16aa1d2cc66632185709_70881_16'